In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
tqdm.pandas()
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
from nltk import tokenize

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk

from tqdm import trange
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from collections import Counter

In [2]:
data = pd.read_csv("Jobs_Law_Sub.csv")
df = data.copy()

In [3]:
df.head()

,JobRefNo,DigerPozisyon,Qualifications,sub-classification
0,3463993,Avukat,<p>GENEL NİTELİKLER VE İŞ TANIMI</p><p>Ünivers...,Avukatlık
1,3463712,Corporate Lawyer,<p><span><span><span>GSI is looking to recruit...,Avukatlık
2,3462810,Avukat,<p><strong>Genel Nitelikler Ve İş Tanımı</stro...,Avukatlık
3,3461644,Avukat Asistanı,<p>Şişli'de bulunan hukuk ofisimizde çalışmak ...,Avukatlık
4,3461014,Associate (Litigation ),"<p class=""x_x_x_x_x_x_x_MsoNormal""><font style...",Avukatlık


In [4]:
df.tail()

,JobRefNo,DigerPozisyon,Qualifications,sub-classification
2495,2811046,İcra Takip Elemanı (İşlemci),<p>İstanbul&nbsp;<strong>Avrupa&nbsp;</strong>...,İcra
2496,2811011,İcra Takip Elemanı,"<p><span>Küresel Hukuk Bürosu olarak, her yıl ...",İcra
2497,2810971,Assistant Legal Counsel,<p><span><strong>QUALIFICATIONS</strong></span...,Hukuk
2498,2810942,Regulatory Assistant Specialist,"<p><span><strong><em>Bilim Pharmaceuticals, wh...",Ruhsatlandırma
2499,2810771,Avukat,<p><br></p><p><strong><span>İŞ TANIMI :</span>...,Avukatlık


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   JobRefNo            2500 non-null   int64 
 1   DigerPozisyon       2500 non-null   object
 2   Qualifications      2500 non-null   object
 3   sub-classification  2500 non-null   object
dtypes: int64(1), object(3)
memory usage: 78.2+ KB


In [6]:
print('Number of Classification=>',len(data['sub-classification'].unique()))
print('Number of DigerPozisyon=>',len(data['DigerPozisyon'].unique()))

Number of Classification=> 14
Number of DigerPozisyon=> 589


In [7]:
def CleanHtml(text):
    return BeautifulSoup(text).getText(' ') 

In [8]:
CleanHtml(BeautifulSoup(df.iloc[2]['Qualifications']).get_text(" "))

'Genel Nitelikler Ve İş Tanımı İzmir’deki hukuk büromuzda çalışmak üzere, \xa0özel hukuk (Özellikle Borçlar Hukuku ve Ticaret Hukuku) alanlarında aktif şekilde en az\xa02\xa0yıl çalışmış, dava süreçlerine ve kurumsal süreçlere hakim, sözleşmelerin incelenmesi, dava takibi yapılması, kurumsal dokümanların hazırlanması konusunda deneyimli avukat aranmaktadır. \xa0 Adayda Aranan Özellikler Üniversitelerin hukuk fakültelerinden mezun, MS Office, Mevzuat Programları ve UYAP’ı iyi düzeyde kullanabilen, Sözlü ve yazılı iletişim becerisi yüksek, Erkek adaylar için askerlik görevini tamamlamış.'

In [9]:
metin = 'Genel Nitelikler Ve İş Tanımı İzmir’deki hukuk büromuzda çalışmak üzere, özel hukuk (Özellikle Borçlar Hukuku ve Ticaret Hukuku) alanlarında aktif şekilde en az 2 yıl çalışmış, dava süreçlerine ve kurumsal süreçlere hakim, sözleşmelerin incelenmesi, dava takibi yapılması, kurumsal dokümanların hazırlanması konusunda deneyimli avukat aranmaktadır. Adayda Aranan Özellikler Üniversitelerin hukuk fakültelerinden mezun, MS Office, Mevzuat Programları ve UYAP’ı iyi düzeyde kullanabilen, Sözlü ve yazılı iletişim becerisi yüksek, Erkek adaylar için askerlik görevini tamamlamış.'

metin = metin.encode('utf-8').decode('utf-8')


In [10]:
import re

metin = re.sub('\s+', ' ', metin)

In [11]:
metin

'Genel Nitelikler Ve İş Tanımı İzmir’deki hukuk büromuzda çalışmak üzere, özel hukuk (Özellikle Borçlar Hukuku ve Ticaret Hukuku) alanlarında aktif şekilde en az 2 yıl çalışmış, dava süreçlerine ve kurumsal süreçlere hakim, sözleşmelerin incelenmesi, dava takibi yapılması, kurumsal dokümanların hazırlanması konusunda deneyimli avukat aranmaktadır. Adayda Aranan Özellikler Üniversitelerin hukuk fakültelerinden mezun, MS Office, Mevzuat Programları ve UYAP’ı iyi düzeyde kullanabilen, Sözlü ve yazılı iletişim becerisi yüksek, Erkek adaylar için askerlik görevini tamamlamış.'

In [12]:
df['Qualifications_HtmlFree'] = data.progress_apply(lambda x: CleanHtml(x['Qualifications']),axis=1)


100%|██████████| 2500/2500 [00:05<00:00, 443.18it/s]


In [13]:
df.head()

,JobRefNo,DigerPozisyon,Qualifications,sub-classification,Qualifications_HtmlFree
0,3463993,Avukat,<p>GENEL NİTELİKLER VE İŞ TANIMI</p><p>Ünivers...,Avukatlık,GENEL NİTELİKLER VE İŞ TANIMI Üniversitelerin ...
1,3463712,Corporate Lawyer,<p><span><span><span>GSI is looking to recruit...,Avukatlık,GSI is looking to recruit an associate lawyer ...
2,3462810,Avukat,<p><strong>Genel Nitelikler Ve İş Tanımı</stro...,Avukatlık,Genel Nitelikler Ve İş Tanımı İzmir’deki hukuk...
3,3461644,Avukat Asistanı,<p>Şişli'de bulunan hukuk ofisimizde çalışmak ...,Avukatlık,Şişli'de bulunan hukuk ofisimizde çalışmak üze...
4,3461014,Associate (Litigation ),"<p class=""x_x_x_x_x_x_x_MsoNormal""><font style...",Avukatlık,"NAZALI Tax and Legal, finans alanında hukuki ç..."


In [14]:
lenght = len(df['Qualifications'][2])
print(f'Length of a sample job description: {lenght}')

Length of a sample job description: 682


In [15]:
df['Length'] = df['Qualifications_HtmlFree'].str.len()
df.head(10)

,JobRefNo,DigerPozisyon,Qualifications,sub-classification,Qualifications_HtmlFree,Length
0,3463993,Avukat,<p>GENEL NİTELİKLER VE İŞ TANIMI</p><p>Ünivers...,Avukatlık,GENEL NİTELİKLER VE İŞ TANIMI Üniversitelerin ...,808
1,3463712,Corporate Lawyer,<p><span><span><span>GSI is looking to recruit...,Avukatlık,GSI is looking to recruit an associate lawyer ...,1226
2,3462810,Avukat,<p><strong>Genel Nitelikler Ve İş Tanımı</stro...,Avukatlık,Genel Nitelikler Ve İş Tanımı İzmir’deki hukuk...,579
3,3461644,Avukat Asistanı,<p>Şişli'de bulunan hukuk ofisimizde çalışmak ...,Avukatlık,Şişli'de bulunan hukuk ofisimizde çalışmak üze...,228
4,3461014,Associate (Litigation ),"<p class=""x_x_x_x_x_x_x_MsoNormal""><font style...",Avukatlık,"NAZALI Tax and Legal, finans alanında hukuki ç...",2306
5,3460989,Avukat,"<p><br></p><ul><li>Hukuk Fakültesinden mezun,<...",Avukatlık,"Hukuk Fakültesinden mezun, Şirket bünyesinde t...",1267
6,3460906,Avukat Çalışma Arkadaşları Aranmaktadır,"<p class=""x_x_x_x_x_MsoNormal"" align=""center"">...",Avukatlık,25 yıldan bu yana ulusal ve uluslararası yerli...,4818
7,3460830,Ticari Tahsilat Avukat,"<p class=""x_MsoNormal""><u>GELECEK VARLIK YÖNET...",Avukatlık,"GELECEK VARLIK YÖNETİMİ , ulusal ve uluslarara...",2096
8,3460727,Yönetici İcra Avukatı,<p>PROJE YÖNETİCİSİ OLARAK İCRA BÖLÜMÜNDE GÖRE...,Avukatlık,PROJE YÖNETİCİSİ OLARAK İCRA BÖLÜMÜNDE GÖREV A...,134
9,3460636,Avukat Asistanı,<p>Şişli'de bulunan hukuk ofisimizde çalışmak ...,Avukatlık,Şişli'de bulunan hukuk ofisimizde çalışmak üze...,228


In [16]:
word_count = df['Qualifications_HtmlFree'][3].split()
print(f'Word count in a sample review: {len(word_count)}')

Word count in a sample review: 30


In [17]:
def word_count(review):
    review_list = review.split()
    return len(review_list)

df['Word_count'] = df['Qualifications_HtmlFree'].progress_apply(word_count)
df.head(3)

100%|██████████| 2500/2500 [00:00<00:00, 13773.04it/s]


,JobRefNo,DigerPozisyon,Qualifications,sub-classification,Qualifications_HtmlFree,Length,Word_count
0,3463993,Avukat,<p>GENEL NİTELİKLER VE İŞ TANIMI</p><p>Ünivers...,Avukatlık,GENEL NİTELİKLER VE İŞ TANIMI Üniversitelerin ...,808,106
1,3463712,Corporate Lawyer,<p><span><span><span>GSI is looking to recruit...,Avukatlık,GSI is looking to recruit an associate lawyer ...,1226,168
2,3462810,Avukat,<p><strong>Genel Nitelikler Ve İş Tanımı</stro...,Avukatlık,Genel Nitelikler Ve İş Tanımı İzmir’deki hukuk...,579,72


In [18]:
df['mean_word_length'] = df['Qualifications_HtmlFree'].map(lambda rev: np.mean([len(word) for word in rev.split()]))
df.head(3)

,JobRefNo,DigerPozisyon,Qualifications,sub-classification,Qualifications_HtmlFree,Length,Word_count,mean_word_length
0,3463993,Avukat,<p>GENEL NİTELİKLER VE İŞ TANIMI</p><p>Ünivers...,Avukatlık,GENEL NİTELİKLER VE İŞ TANIMI Üniversitelerin ...,808,106,6.632075
1,3463712,Corporate Lawyer,<p><span><span><span>GSI is looking to recruit...,Avukatlık,GSI is looking to recruit an associate lawyer ...,1226,168,6.202381
2,3462810,Avukat,<p><strong>Genel Nitelikler Ve İş Tanımı</stro...,Avukatlık,Genel Nitelikler Ve İş Tanımı İzmir’deki hukuk...,579,72,7.013889


In [19]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KARACIK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
df['mean_sent_length'] = df['Qualifications_HtmlFree'].map(lambda rev: np.mean([len(sent) for sent in sent_tokenize(rev)]))
df.head(3)


,JobRefNo,DigerPozisyon,Qualifications,sub-classification,Qualifications_HtmlFree,Length,Word_count,mean_word_length,mean_sent_length
0,3463993,Avukat,<p>GENEL NİTELİKLER VE İŞ TANIMI</p><p>Ünivers...,Avukatlık,GENEL NİTELİKLER VE İŞ TANIMI Üniversitelerin ...,808,106,6.632075,808.000000
1,3463712,Corporate Lawyer,<p><span><span><span>GSI is looking to recruit...,Avukatlık,GSI is looking to recruit an associate lawyer ...,1226,168,6.202381,80.066667
2,3462810,Avukat,<p><strong>Genel Nitelikler Ve İş Tanımı</stro...,Avukatlık,Genel Nitelikler Ve İş Tanımı İzmir’deki hukuk...,579,72,7.013889,288.000000


In [21]:
df = df.rename(columns={'sub-classification': 'subclassification'})


In [22]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df.subclassification)
df['categorical_label'] = le.transform(df.subclassification)
df['categorical_label'] = df['categorical_label'].astype(int)
df.head(2)

,JobRefNo,DigerPozisyon,Qualifications,subclassification,Qualifications_HtmlFree,Length,Word_count,mean_word_length,mean_sent_length,categorical_label
0,3463993,Avukat,<p>GENEL NİTELİKLER VE İŞ TANIMI</p><p>Ünivers...,Avukatlık,GENEL NİTELİKLER VE İŞ TANIMI Üniversitelerin ...,808,106,6.632075,808.000000,1
1,3463712,Corporate Lawyer,<p><span><span><span>GSI is looking to recruit...,Avukatlık,GSI is looking to recruit an associate lawyer ...,1226,168,6.202381,80.066667,1


In [23]:
def clean(review):
    
    review = re.sub(r'(?<![a-zA-ZçÇğĞıİöÖşŞüÜ])\w+|\w+(?![a-zA-ZçÇğĞıİöÖşŞüÜ])', ' ', review)
    review = review.lower()
    review = review.replace('&nbsp;', ' ').replace('“', ' ').replace('·', ' ').replace('i̇','i')
    review = review.replace('•', ' ').replace('\xa0', ' ')
    review = review.replace('”', ' ').replace('nan', '').replace('\r', '').replace('’', ' ')
    review = " ".join([word for word in review.split() if word not in stopwordList])
    
    return review

In [24]:
def tokenize(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r"#(\w+)", '', text)
    text = re.sub(r"@(\w+)", '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip().lower()
    text = text.replace('&nbsp;', ' ').replace('“', ' ').replace('·', ' ').replace('i̇','i')
    text = text.replace('•', ' ').replace('\xa0', ' ')
    text = text.replace('”', ' ').replace('nan', '').replace('\r', '').replace('’', ' ')
    text = " ".join([word for word in text.split() if word not in stopwordList])
    return text

In [25]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stopwordList = set(stopwords.words('english'))

def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stopwordList]
    return tokens

df['Clean'] = df['Qualifications_HtmlFree'].apply(tokenize)
df.head(3)



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KARACIK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,JobRefNo,DigerPozisyon,Qualifications,subclassification,Qualifications_HtmlFree,Length,Word_count,mean_word_length,mean_sent_length,categorical_label,Clean
0,3463993,Avukat,<p>GENEL NİTELİKLER VE İŞ TANIMI</p><p>Ünivers...,Avukatlık,GENEL NİTELİKLER VE İŞ TANIMI Üniversitelerin ...,808,106,6.632075,808.000000,1,"[genel, ni̇teli̇kler, i̇ş, tanimi, üniversitel..."
1,3463712,Corporate Lawyer,<p><span><span><span>GSI is looking to recruit...,Avukatlık,GSI is looking to recruit an associate lawyer ...,1226,168,6.202381,80.066667,1,"[gsi, looking, recruit, associate, lawyer, joi..."
2,3462810,Avukat,<p><strong>Genel Nitelikler Ve İş Tanımı</stro...,Avukatlık,Genel Nitelikler Ve İş Tanımı İzmir’deki hukuk...,579,72,7.013889,288.000000,1,"[genel, nitelikler, i̇ş, tanımı, i̇zmir, deki,..."


In [26]:
def corpus(text):
    text_list = word_tokenize(text)
    return text_list


df['Clean'] = df['Clean'].apply(lambda x: ' '.join(x))

df.head(3)


,JobRefNo,DigerPozisyon,Qualifications,subclassification,Qualifications_HtmlFree,Length,Word_count,mean_word_length,mean_sent_length,categorical_label,Clean
0,3463993,Avukat,<p>GENEL NİTELİKLER VE İŞ TANIMI</p><p>Ünivers...,Avukatlık,GENEL NİTELİKLER VE İŞ TANIMI Üniversitelerin ...,808,106,6.632075,808.000000,1,genel ni̇teli̇kler i̇ş tanimi üniversitelerin ...
1,3463712,Corporate Lawyer,<p><span><span><span>GSI is looking to recruit...,Avukatlık,GSI is looking to recruit an associate lawyer ...,1226,168,6.202381,80.066667,1,gsi looking recruit associate lawyer join high...
2,3462810,Avukat,<p><strong>Genel Nitelikler Ve İş Tanımı</stro...,Avukatlık,Genel Nitelikler Ve İş Tanımı İzmir’deki hukuk...,579,72,7.013889,288.000000,1,genel nitelikler i̇ş tanımı i̇zmir deki hukuk ...


In [27]:
df['Clean_lists'] = df['Clean'].apply(corpus)
df.head(3)

,JobRefNo,DigerPozisyon,Qualifications,subclassification,Qualifications_HtmlFree,Length,Word_count,mean_word_length,mean_sent_length,categorical_label,Clean,Clean_lists
0,3463993,Avukat,<p>GENEL NİTELİKLER VE İŞ TANIMI</p><p>Ünivers...,Avukatlık,GENEL NİTELİKLER VE İŞ TANIMI Üniversitelerin ...,808,106,6.632075,808.000000,1,genel ni̇teli̇kler i̇ş tanimi üniversitelerin ...,"[genel, ni̇teli̇kler, i̇ş, tanimi, üniversitel..."
1,3463712,Corporate Lawyer,<p><span><span><span>GSI is looking to recruit...,Avukatlık,GSI is looking to recruit an associate lawyer ...,1226,168,6.202381,80.066667,1,gsi looking recruit associate lawyer join high...,"[gsi, looking, recruit, associate, lawyer, joi..."
2,3462810,Avukat,<p><strong>Genel Nitelikler Ve İş Tanımı</stro...,Avukatlık,Genel Nitelikler Ve İş Tanımı İzmir’deki hukuk...,579,72,7.013889,288.000000,1,genel nitelikler i̇ş tanımı i̇zmir deki hukuk ...,"[genel, nitelikler, i̇ş, tanımı, i̇zmir, deki,..."


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


def ngram_vectorize(train_texts, train_labels, val_texts, NGRAM_RANGE = (1, 2),TOP_K = 20000
                    ,TOKEN_MODE = 'word',MIN_DOCUMENT_FREQUENCY = 2, vForm = 'tfidf'):

    kwargs = {
            'ngram_range': NGRAM_RANGE,  # Use 1-grams + 2-grams.
            'dtype': 'int32',
            'strip_accents': 'unicode',
            'decode_error': 'replace',
            'analyzer': TOKEN_MODE,  # Split text into word tokens.
            'min_df': MIN_DOCUMENT_FREQUENCY,
    }

    if vForm == 'tfidf':
        vectorizer = TfidfVectorizer(**kwargs)
    elif vForm == 'count':
        vectorizer = CountVectorizer(**kwargs) 
    else:
        print('Invalid vectorization form')
        return

   
    x_train = vectorizer.fit_transform(train_texts)

  
    x_val = vectorizer.transform(val_texts)

   
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
    selector.fit(x_train, train_labels)
    x_train = selector.transform(x_train).astype('float32')
    x_val = selector.transform(x_val).astype('float32')
    return x_train, x_val,vectorizer

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.metrics import accuracy_score, auc, roc_auc_score
import xgboost as xgb

In [30]:
from sklearn.model_selection import train_test_split
from collections import Counter

class_counts = Counter(df['categorical_label'])
stratify_list = [class_counts[y] if class_counts[y] >= 2 else None for y in df['categorical_label']]

x_train, x_test, y_train, y_test, indices_train, indices_test = train_test_split(df['Clean'], 
                                                    df['categorical_label'], 
                                                    df.index,
                                                    test_size=0.1, 
                                                    random_state=42)


print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)





(2250,) (250,) (2250,) (250,)


In [31]:
xtrain_tfv,xvalid_tfv,vectorizer = ngram_vectorize(x_train, y_train, x_test, vForm = 'tfidf',  NGRAM_RANGE = (1, 2),
                                        TOP_K = 20000, TOKEN_MODE = 'word',MIN_DOCUMENT_FREQUENCY = 2)

C:\Users\KARACIK\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:2070: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  warnings.warn(


In [32]:
df.isnull().values.any()

False

In [33]:
df[df['categorical_label'].isnull()]

,JobRefNo,DigerPozisyon,Qualifications,subclassification,Qualifications_HtmlFree,Length,Word_count,mean_word_length,mean_sent_length,categorical_label,Clean,Clean_lists


In [34]:
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

In [35]:
lr_word = LogisticRegression(solver='sag', verbose=1)
lr_word.fit(xtrain_tfv, y_train)
y_pred_word = lr_word.predict(xvalid_tfv)
LR_results=accuracy_score(y_test, y_pred_word)
results = {'LR_TfIDF_unigram': LR_results} 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 45 epochs took 2 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s finished


In [36]:
results

{'LR_TfIDF_unigram': 0.752}

In [37]:
df.columns

Index(['JobRefNo', 'DigerPozisyon', 'Qualifications', 'subclassification',
       'Qualifications_HtmlFree', 'Length', 'Word_count', 'mean_word_length',
       'mean_sent_length', 'categorical_label', 'Clean', 'Clean_lists'],
      dtype='object')

In [38]:
ngram_vectorize_output = ngram_vectorize(x_train, y_train, x_test, vForm='count', NGRAM_RANGE=(1,3), 
                                         TOP_K=20000, TOKEN_MODE='word', MIN_DOCUMENT_FREQUENCY=2)
xtrain_tfv, xvalid_tfv, feature_names = ngram_vectorize(x_train, y_train, x_test, vForm='tfidf')
lr_word.fit(xtrain_tfv, y_train)
y_pred_word = lr_word.predict(xvalid_tfv)
LR_results = accuracy_score(y_test, y_pred_word)
results['LR_CountV_trigram'] = LR_results


C:\Users\KARACIK\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:2070: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 46 epochs took 1 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s finished


In [39]:
results

{'LR_TfIDF_unigram': 0.752, 'LR_CountV_trigram': 0.752}

In [40]:
xtrain_tfv,xvalid_tfv,vectorizer = ngram_vectorize(x_train, y_train, x_test, vForm = 'tfidf',  NGRAM_RANGE = (1, 1),
                                        TOP_K = 20000, TOKEN_MODE = 'word',MIN_DOCUMENT_FREQUENCY = 2)
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), y_train)
y_pred_word = clf.predict(xvalid_tfv.tocsc())
LR_results=accuracy_score(y_test, y_pred_word)
results ['XGB_TfIdf_uni']= LR_results


C:\Users\KARACIK\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:2070: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  warnings.warn(


In [41]:
results

{'LR_TfIDF_unigram': 0.752, 'LR_CountV_trigram': 0.752, 'XGB_TfIdf_uni': 0.852}